In [ ]:
import os
import cv2
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
print("Tensorflow version:", tf.__version__)

In [ ]:

images_train = []
labels_train = []
for filename in os.listdir('training/angry'):
    input_image = cv2.imread('training/angry/' + filename)
    if input_image is not None:
        gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
        imgdata = np.array(gray)
        images_train.append(imgdata)
        labels_train.append([1, 0, 0, 0])
for filename in os.listdir('training/happy'):
    input_image = cv2.imread('training/happy/' + filename)
    if input_image is not None:
        gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
        imgdata = np.array(gray)
        images_train.append(imgdata)
        labels_train.append([0, 1, 0, 0])
for filename in os.listdir('training/neutral'):
    input_image = cv2.imread('training/neutral/' + filename)
    if input_image is not None:
        gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
        imgdata = np.array(gray)
        images_train.append(imgdata)
        labels_train.append([0, 0, 1, 0])
for filename in os.listdir('training/sad'):
    input_image = cv2.imread('training/sad/' + filename)
    if input_image is not None:
        gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
        imgdata = np.array(gray)
        images_train.append(imgdata)
        labels_train.append([0, 0, 0, 1])
print(labels_train)

In [ ]:
fig=plt.figure(figsize=(20,20))

nrows = 2
ncols = 5
happy = 0
neutral = 0
angry = 0
sad = 0

for i in range (1, nrows*ncols+1):
    index = np.random.randint(0, len(images_train))
    img = images_train[index]
    if labels_train[index][0] == 1:
        title = 'Angry'
        angry += 1
    elif labels_train[index][1] == 1:
        title = 'Happy'
        happy += 1
    elif labels_train[index][2] == 1:
        title = 'Neutral'
        neutral += 1
    elif labels_train[index][3] == 1:
        title = 'Sad'
        sad += 1

    if i < 6:
        fig.add_subplot(1,ncols,i)
    else:
        fig.add_subplot(2,ncols,i)
    plt.imshow(img, cmap='gray')
    plt.title(title)

plt.show()

In [ ]:
print('Angry: ' + str(angry))
print('Happy: ' + str(happy))
print('Neutral: ' + str(neutral))
print('Sad: ' + str(sad))

In [ ]:

x_train, x_validation, y_train, y_validation = train_test_split(images_train, labels_train, test_size=0.1, random_state=1)

print(y_train)

x_train = np.asarray(x_train)
x_train = np.expand_dims(x_train, axis=0)
x_train = x_train.reshape(x_train.shape[1], x_train.shape[2], x_train.shape[3], x_train.shape[0])
y_train = np.asarray(y_train)

x_validation = np.asarray(x_validation)
x_validation = np.expand_dims(x_validation, axis=0)
x_validation = x_validation.reshape(x_validation.shape[1], x_validation.shape[2], x_validation.shape[3], x_validation.shape[0])
y_validation = np.asarray(y_validation)

In [ ]:
# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(48, (3, 3), padding='same', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(4, activation='softmax'))

opt = Adam(learning_rate=0.0005)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy'])
model.summary()

In [ ]:
%%time
epochs = 20
history = model.fit(
    x_train,
    y_train,
    epochs = epochs,
    validation_data = (x_validation, y_validation)
)

In [ ]:
x_test = np.asarray(images_train)
x_test = np.expand_dims(x_test, axis=0)
x_test = x_test.reshape(x_test.shape[1], x_test.shape[2], x_test.shape[3], x_test.shape[0])
y_test = np.asarray(labels_train)

In [ ]:
sample_images = x_test[1].reshape(48, 48)
plt.imshow(sample_images)

In [ ]:
result = model.predict(x_test)
np.argmax(result[1])

In [ ]:
check_result = []
result_sum = 0
for result in model.predict(x_test):
    check_result.append(result)
    result_sum += np.argmax(result)
result_sum
check_result

In [ ]:
model_json = model.to_json()
with open("utils/model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save('utils/model_weights.h5')